In [89]:
import sqlite3
import pandas as pd

def result_of_sql(sql, print_result=True):
    db_path = '../data/Chinook_Sqlite.sqlite'
    with sqlite3.connect(db_path) as conn:
        result = pd.read_sql_query(sql, conn)
        if print_result:
            print(result)
        return result

def table_info(table_name):
    sql = f"PRAGMA table_info('{table_name}');"
    print(f"Table: {table_name}")
    result_of_sql(sql)

In [90]:
sql = """
SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;
"""
result = result_of_sql(sql)

             name
0           Album
1          Artist
2        Customer
3        Employee
4           Genre
5         Invoice
6     InvoiceLine
7       MediaType
8        Playlist
9   PlaylistTrack
10          Track


In [91]:
table_names = result['name'].to_list()
for table_name in table_names:
    print(table_name)

Album
Artist
Customer
Employee
Genre
Invoice
InvoiceLine
MediaType
Playlist
PlaylistTrack
Track


In [92]:
table_names = result['name'].to_list()
for table_name in table_names:
    table_info(table_name)

Table: Album
   cid      name           type  notnull dflt_value  pk
0    0   AlbumId        INTEGER        1       None   1
1    1     Title  NVARCHAR(160)        1       None   0
2    2  ArtistId        INTEGER        1       None   0
Table: Artist
   cid      name           type  notnull dflt_value  pk
0    0  ArtistId        INTEGER        1       None   1
1    1      Name  NVARCHAR(120)        0       None   0
Table: Customer
    cid          name          type  notnull dflt_value  pk
0     0    CustomerId       INTEGER        1       None   1
1     1     FirstName  NVARCHAR(40)        1       None   0
2     2      LastName  NVARCHAR(20)        1       None   0
3     3       Company  NVARCHAR(80)        0       None   0
4     4       Address  NVARCHAR(70)        0       None   0
5     5          City  NVARCHAR(40)        0       None   0
6     6         State  NVARCHAR(40)        0       None   0
7     7       Country  NVARCHAR(40)        0       None   0
8     8    PostalCode  NV

# Questions

1. Number of Tracks are longer than 1 Million Millisecond 
2. For Each Genre, how many tracks are there
3. Which Artist has the highest number of tracks and how many do they have
4. Which Track sold the most and how many copies, what the total revenue from sale of that track (sql+pandas)
5. which Album in total is the biggest in Size ( Track - Bytes)
6. Explore MediaType 
7. Number of employees and Number of customers
8. which customer got the most number of tracks, what tracks did they get. 

### number of Tracks are longer than 1 Million Millisecond

In [ ]:
table_info("Track")

sql = "select count(TrackId) from Track where Milliseconds > 1000000"
result_of_sql(sql)

Table: Track
   cid          name           type  notnull dflt_value  pk
0    0       TrackId        INTEGER        1       None   1
1    1          Name  NVARCHAR(200)        1       None   0
2    2       AlbumId        INTEGER        0       None   0
3    3   MediaTypeId        INTEGER        1       None   0
4    4       GenreId        INTEGER        0       None   0
5    5      Composer  NVARCHAR(220)        0       None   0
6    6  Milliseconds        INTEGER        1       None   0
7    7         Bytes        INTEGER        0       None   0
8    8     UnitPrice  NUMERIC(10,2)        1       None   0
   count(TrackId)
0             215


,count(TrackId)
0,215


### For Each Genre, How many tracks are there

In [94]:
# For Each Genre, How many tracks are there
table_info('Genre')

sql = """
select g.name, count(t.TrackId) as TotalCount from Track t
JOIN Genre as g
ON t.GenreId = g.GenreId
GROUP BY g.name
ORDER BY TotalCount DESC
"""
result_of_sql(sql,print_result=False)

Table: Genre
   cid     name           type  notnull dflt_value  pk
0    0  GenreId        INTEGER        1       None   1
1    1     Name  NVARCHAR(120)        0       None   0


,Name,TotalCount
0,Rock,1297
1,Latin,579
2,Metal,374
3,Alternative & Punk,332
4,Jazz,130
5,TV Shows,93
6,Blues,81
7,Classical,74
8,Drama,64
9,R&B/Soul,61


In [95]:
# Which Artist has the highest number of tracks and How many they do have
table_info('Artist')
table_info('Track')
table_info('Album')

Table: Artist
   cid      name           type  notnull dflt_value  pk
0    0  ArtistId        INTEGER        1       None   1
1    1      Name  NVARCHAR(120)        0       None   0
Table: Track
   cid          name           type  notnull dflt_value  pk
0    0       TrackId        INTEGER        1       None   1
1    1          Name  NVARCHAR(200)        1       None   0
2    2       AlbumId        INTEGER        0       None   0
3    3   MediaTypeId        INTEGER        1       None   0
4    4       GenreId        INTEGER        0       None   0
5    5      Composer  NVARCHAR(220)        0       None   0
6    6  Milliseconds        INTEGER        1       None   0
7    7         Bytes        INTEGER        0       None   0
8    8     UnitPrice  NUMERIC(10,2)        1       None   0
Table: Album
   cid      name           type  notnull dflt_value  pk
0    0   AlbumId        INTEGER        1       None   1
1    1     Title  NVARCHAR(160)        1       None   0
2    2  ArtistId        

In [96]:
sql = """
select a.name, count(t.TrackId) as TrackCount from Track t
JOIN Album al
ON t.AlbumId = al.AlbumId
JOIN Artist a
ON  al.ArtistId = a.ArtistId
GROUP BY a.name
ORDER BY TrackCount DESC
"""
result_of_sql(sql,print_result=False)

,Name,TrackCount
0,Iron Maiden,213
1,U2,135
2,Led Zeppelin,114
3,Metallica,112
4,Lost,92
...,...,...
199,"Academy of St. Martin in the Fields, Sir Nevil...",1
200,"Academy of St. Martin in the Fields, John Birc...",1
201,Academy of St. Martin in the Fields Chamber En...,1
202,Aaron Goldberg,1


### Which Track sold the most and how many copies, what the total revenue from sale of that track (sql+pandas)

In [97]:
# Which Track sold the most and how many copies, what the total revenue from sale of that track (sql+pandas)
#table_info('Track')
table_info('Invoice')
table_info('InvoiceLine')

sql = """
select TrackId, sum(Quantity) as TotalCount from InvoiceLine
GROUP BY TrackId
ORDER BY TotalCount DESC
"""

result = result_of_sql(sql, print_result=False)
print(result.head())

sql = """ 
select i.InvoiceId, il.TrackId, il.UnitPrice, i.Total from Invoice i
JOIN InvoiceLine il
ON i.InvoiceId = il.InvoiceId
Where i.InvoiceId = 1
"""
result = result_of_sql(sql, print_result=False)
print(result.head())

sql = """
select il.TrackId, count(il.InvoiceId) as TotalCount, sum(il.UnitPrice * il.Quantity) as TotalSales from InvoiceLine il
JOIN Invoice i
ON il.InvoiceId = i.InvoiceId
GROUP BY il.TrackId
ORDER BY TotalCount DESC
"""
result_of_sql(sql,print_result=False)


Table: Invoice
   cid               name           type  notnull dflt_value  pk
0    0          InvoiceId        INTEGER        1       None   1
1    1         CustomerId        INTEGER        1       None   0
2    2        InvoiceDate       DATETIME        1       None   0
3    3     BillingAddress   NVARCHAR(70)        0       None   0
4    4        BillingCity   NVARCHAR(40)        0       None   0
5    5       BillingState   NVARCHAR(40)        0       None   0
6    6     BillingCountry   NVARCHAR(40)        0       None   0
7    7  BillingPostalCode   NVARCHAR(10)        0       None   0
8    8              Total  NUMERIC(10,2)        1       None   0
Table: InvoiceLine
   cid           name           type  notnull dflt_value  pk
0    0  InvoiceLineId        INTEGER        1       None   1
1    1      InvoiceId        INTEGER        1       None   0
2    2        TrackId        INTEGER        1       None   0
3    3      UnitPrice  NUMERIC(10,2)        1       None   0
4    4     

,TrackId,TotalCount,TotalSales
0,2,2,1.98
1,8,2,1.98
2,9,2,1.98
3,20,2,1.98
4,32,2,1.98
...,...,...,...
1979,3492,1,0.99
1980,3493,1,0.99
1981,3494,1,0.99
1982,3496,1,0.99


### which Album in total is the biggest in Size ( Track - Bytes)

In [ ]:
table_info('Album')
table_info('Track')

sql = """ 
select a.Title, sum(t.Bytes) as TotalBytes from Album a
JOIN Track t
ON a.AlbumId = t.AlbumId
GROUP BY a.Title
ORDER BY TotalBytes DESC 
"""
result_of_sql(sql, print_result=False)

Table: Album
   cid      name           type  notnull dflt_value  pk
0    0   AlbumId        INTEGER        1       None   1
1    1     Title  NVARCHAR(160)        1       None   0
2    2  ArtistId        INTEGER        1       None   0
Table: Track
   cid          name           type  notnull dflt_value  pk
0    0       TrackId        INTEGER        1       None   1
1    1          Name  NVARCHAR(200)        1       None   0
2    2       AlbumId        INTEGER        0       None   0
3    3   MediaTypeId        INTEGER        1       None   0
4    4       GenreId        INTEGER        0       None   0
5    5      Composer  NVARCHAR(220)        0       None   0
6    6  Milliseconds        INTEGER        1       None   0
7    7         Bytes        INTEGER        0       None   0
8    8     UnitPrice  NUMERIC(10,2)        1       None   0


,Title,TotalBytes
0,"Lost, Season 3",13917603291
1,"Battlestar Galactica (Classic), Season 1",12872621850
2,"Lost, Season 2",12344960921
3,"Heroes, Season 1",11781321607
4,"Battlestar Galactica, Season 3",10059916535
...,...,...
342,Handel: Music for the Royal Fireworks (Origina...,2193734
343,Bizet: Carmen Highlights,2189002
344,Bach: Goldberg Variations,2081895
345,"Charpentier: Divertissements, Airs & Concerts",1973559


### Explore MediaType

In [ ]:
table_info('MediaType')

sql = """ 
select distinct Name from MediaType
"""
result_of_sql(sql)
sql = """ 
select count(*) from MediaType
"""
result_of_sql(sql)
sql = """ 
select m.Name, count(t.TrackId) as TrackCount from Track t
JOIN MediaType m
ON t.MediaTypeId = m.MediaTypeId
GROUP BY t.MediaTypeId
ORDER BY TrackCount DESC 
"""
result_of_sql(sql, print_result=False)

Table: MediaType
   cid         name           type  notnull dflt_value  pk
0    0  MediaTypeId        INTEGER        1       None   1
1    1         Name  NVARCHAR(120)        0       None   0
                          Name
0              MPEG audio file
1     Protected AAC audio file
2  Protected MPEG-4 video file
3     Purchased AAC audio file
4               AAC audio file
   count(*)
0         5


,Name,TrackCount
0,MPEG audio file,3034
1,Protected AAC audio file,237
2,Protected MPEG-4 video file,214
3,AAC audio file,11
4,Purchased AAC audio file,7



### Number of employees and Number of customers

In [ ]:
table_info('Employee')
table_info('Customer')

Table: Employee
    cid        name          type  notnull dflt_value  pk
0     0  EmployeeId       INTEGER        1       None   1
1     1    LastName  NVARCHAR(20)        1       None   0
2     2   FirstName  NVARCHAR(20)        1       None   0
3     3       Title  NVARCHAR(30)        0       None   0
4     4   ReportsTo       INTEGER        0       None   0
5     5   BirthDate      DATETIME        0       None   0
6     6    HireDate      DATETIME        0       None   0
7     7     Address  NVARCHAR(70)        0       None   0
8     8        City  NVARCHAR(40)        0       None   0
9     9       State  NVARCHAR(40)        0       None   0
10   10     Country  NVARCHAR(40)        0       None   0
11   11  PostalCode  NVARCHAR(10)        0       None   0
12   12       Phone  NVARCHAR(24)        0       None   0
13   13         Fax  NVARCHAR(24)        0       None   0
14   14       Email  NVARCHAR(60)        0       None   0
Table: Customer
    cid          name          type  not

In [101]:
sql = """ 
select count(EmployeeId) from Employee
"""
result_of_sql(sql)
sql =""" 
select count(CustomerId) from Customer
"""
result_of_sql(sql, print_result=False)

   count(EmployeeId)
0                  8


,count(CustomerId)
0,59


### which customer got the most number of tracks, what tracks did they get. 

In [ ]:
table_info('Customer')
table_info('Invoice')
table_info('InvoiceLine')


Table: Customer
    cid          name          type  notnull dflt_value  pk
0     0    CustomerId       INTEGER        1       None   1
1     1     FirstName  NVARCHAR(40)        1       None   0
2     2      LastName  NVARCHAR(20)        1       None   0
3     3       Company  NVARCHAR(80)        0       None   0
4     4       Address  NVARCHAR(70)        0       None   0
5     5          City  NVARCHAR(40)        0       None   0
6     6         State  NVARCHAR(40)        0       None   0
7     7       Country  NVARCHAR(40)        0       None   0
8     8    PostalCode  NVARCHAR(10)        0       None   0
9     9         Phone  NVARCHAR(24)        0       None   0
10   10           Fax  NVARCHAR(24)        0       None   0
11   11         Email  NVARCHAR(60)        1       None   0
12   12  SupportRepId       INTEGER        0       None   0
Table: Invoice
   cid               name           type  notnull dflt_value  pk
0    0          InvoiceId        INTEGER        1       None   1

In [103]:
sql = """ 
select c.CustomerId, count(il.TrackId) as NumberOfTracks from Invoice i
JOIN Customer c
ON i.CustomerId = c.CustomerId
JOIN InvoiceLine il
ON i.InvoiceId = il.InvoiceId
GROUP BY i.CustomerId
ORDER BY NumberOfTracks DESC
"""
result_of_sql(sql)
sql = """ 
select c.CustomerId, CONCAT(c.FirstName, ' ', c.LastName) AS CustomerName, il.TrackId, t.Name from Invoice i
JOIN Customer c
ON i.CustomerId = c.CustomerId
JOIN InvoiceLine il
ON i.InvoiceId = il.InvoiceId
JOIN Track t
ON t.TrackId = il.TrackId
Where i.CustomerId = 1
ORDER BY il.TrackId
"""
result_of_sql(sql, print_result=False)

    CustomerId  NumberOfTracks
0            1              38
1            2              38
2            3              38
3            4              38
4            5              38
5            6              38
6            7              38
7            8              38
8            9              38
9           10              38
10          11              38
11          12              38
12          13              38
13          14              38
14          15              38
15          16              38
16          17              38
17          18              38
18          19              38
19          20              38
20          21              38
21          22              38
22          23              38
23          24              38
24          25              38
25          26              38
26          27              38
27          28              38
28          29              38
29          30              38
30          31              38
31      

,CustomerId,CustomerName,TrackId,Name
0,1,Luís Gonçalves,262,Interlude Zumbi
1,1,Luís Gonçalves,271,Rios Pontes & Overdrives
2,1,Luís Gonçalves,280,Lixo Do Mangue
3,1,Luís Gonçalves,289,Podes Crer
4,1,Luís Gonçalves,298,A Cor Do Sol
5,1,Luís Gonçalves,307,Doutor
6,1,Luís Gonçalves,316,Linha Do Equador
7,1,Luís Gonçalves,325,TriboTchan
8,1,Luís Gonçalves,334,Reggae Tchan
9,1,Luís Gonçalves,343,Communication Breakdown(2)
